In [1]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent.absolute()))
import config

In [2]:
from context import Context
from utils.utils_chain import WrapperAddress as Address, Account, hex_to_string
from utils.utils_chain import nominated_amount

context = Context()

2023-08-09 19:29:50,484 - WARNING - Structure definition for egld_wraps not found in /home/ovidiu/projects/dex/mx-sdk-py-exchange/deploy/configs-mainnet/deploy_structure.json
2023-08-09 19:29:50,488 - WARNING - Structure definition for stakings_boosted not found in /home/ovidiu/projects/dex/mx-sdk-py-exchange/deploy/configs-mainnet/deploy_structure.json
2023-08-09 19:29:50,489 - WARNING - Structure definition for metastakings_boosted not found in /home/ovidiu/projects/dex/mx-sdk-py-exchange/deploy/configs-mainnet/deploy_structure.json


loaded 10 accounts from 1 PEM files.


2023-08-09 19:29:51,945 - DEBUG - Account.sync_nonce() done: 3298
2023-08-09 19:29:53,061 - DEBUG - Account.sync_nonce() done: 1
2023-08-09 19:29:53,063 - INFO - Loaded 10 tokens.
2023-08-09 19:29:55,143 - DEBUG - Account.sync_nonce() done: 3298
2023-08-09 19:29:55,144 - INFO - Loaded 0 egld_wraps.
2023-08-09 19:29:55,146 - INFO - Loaded 1 locked_assets.
2023-08-09 19:29:55,148 - INFO - Loaded 1 proxies.
2023-08-09 19:29:55,150 - INFO - Loaded 0 simple_locks.
2023-08-09 19:29:55,152 - INFO - Loaded 1 simple_locks_energy.
2023-08-09 19:29:55,153 - INFO - Loaded 1 fees_collectors.
2023-08-09 19:29:55,155 - INFO - Loaded 1 unstakers.
2023-08-09 19:29:55,156 - INFO - Loaded 1 proxies_v2.
2023-08-09 19:29:55,157 - INFO - Loaded 0 router.
2023-08-09 19:29:55,159 - INFO - Loaded 1 router_v2.
2023-08-09 19:29:55,160 - INFO - Loaded 0 pairs.
2023-08-09 19:29:55,162 - INFO - Loaded 9 pairs_v2.
2023-08-09 19:29:55,163 - INFO - Loaded 0 farms_community.
2023-08-09 19:29:55,164 - INFO - Loaded 0 fa

In [3]:
from utils.contract_retrievers import retrieve_farm_by_address
from utils.utils_chain import get_token_details_for_address, get_all_token_nonces_details_for_account
from utils.utils_tx import ESDTToken
from contracts.pair_contract import PairContract
from contracts.farm_contract import FarmContract, ClaimRewardsFarmEvent, EnterFarmEvent
from contracts.dex_proxy_contract import DexProxyContract, DexProxyClaimRewardsEvent, DexProxyEnterFarmEvent, DexProxyExitFarmEvent, DexProxyRemoveLiquidityEvent, DexProxyAddLiquidityEvent
from contracts.simple_lock_energy_contract import SimpleLockEnergyContract

In [4]:
farm_contract: FarmContract
farm_contract = context.get_contracts(config.FARMS_V2)[0]
proxy_contract: DexProxyContract
proxy_contract = context.get_contracts(config.PROXIES_V2)[0]
pair_contract: PairContract
pair_contract = context.get_contracts(config.PAIRS_V2)[0]
energy_contract: SimpleLockEnergyContract
energy_contract = context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0]

In [5]:
user = Account(pem_file="~/Documents/sh1.pem")
user.address = Address("erd1ynecckmk8apsy086e49ng6mske566e8c5sc9val809z7rcth4j2sdal443")
user.sync_nonce(context.network_provider.proxy)

2023-08-09 19:29:56,584 - DEBUG - Account.sync_nonce() done: 426


claim proxy farm rewards

In [ ]:
farm_token_nonce, farm_token_amount, _ = get_token_details_for_address(proxy_contract.proxy_farm_token, user.address.bech32(), context.network_provider.proxy)
if farm_token_nonce == 0:
    print("No farm tokens")
    exit()
event = DexProxyClaimRewardsEvent(farm_contract, proxy_contract.proxy_farm_token, farm_token_nonce, farm_token_amount)
txhash = proxy_contract.claim_rewards_proxy(user, context.network_provider.proxy, event)

2023-08-08 17:13:25,007 - DEBUG - Account.sync_nonce() done: 405
2023-08-08 17:13:27,683 - DEBUG - Executing claim rewards via proxy for user erd1ynecckmk8apsy086e49ng6mske566e8c5sc9val809z7rcth4j2sdal443 with event {'farmContract': <contracts.farm_contract.FarmContract object at 0x7f774c7db790>, 'token': 'XMEXFARM-e0a3dc', 'nonce': 190622, 'amount': 57629834055703580649}
2023-08-08 17:13:27,685 - DEBUG - claim rewards via proxy
2023-08-08 17:13:27,848 - DEBUG - Contract call arguments: [erd1qqqqqqqqqqqqqpgqapxdp9gjxtg60mjwhle3n6h88zch9e7kkp2s8aqhkg]
2023-08-08 17:13:29,468 - INFO - No explorer known for https://proxy-shadowfork-one.elrond.ro. transaction raw path: transactions/1c8fe88e85d0b4aaca2974ffdf110a34a891eae6d82b6b4db6d5660c8b838d0c


exit position

In [23]:
farm_token_nonce, farm_token_amount, _ = get_token_details_for_address(proxy_contract.proxy_farm_token, user.address.bech32(), context.network_provider.proxy)
if farm_token_nonce == 0:
    print("No farm tokens")
    exit()
event = DexProxyExitFarmEvent(farm_contract, proxy_contract.proxy_farm_token, farm_token_nonce, farm_token_amount // 2)
txhash = proxy_contract.exit_farm_proxy(user, context.network_provider.proxy, event)

2023-08-11 01:43:25,084 - DEBUG - Executing exit farm via proxy for user erd1ynecckmk8apsy086e49ng6mske566e8c5sc9val809z7rcth4j2sdal443 with event {'farmContract': <contracts.farm_contract.FarmContract object at 0x7f9dbe883310>, 'token': 'XMEXFARM-e0a3dc', 'nonce': 193574, 'amount': 27219538694909253357}
2023-08-11 01:43:25,086 - DEBUG - exit farm via proxy
2023-08-11 01:43:25,497 - DEBUG - Contract call arguments: [erd1qqqqqqqqqqqqqpgqapxdp9gjxtg60mjwhle3n6h88zch9e7kkp2s8aqhkg, 27219538694909253357]
2023-08-11 01:43:25,807 - INFO - No explorer known for https://proxy-shadowfork-one.elrond.ro. transaction raw path: transactions/805257d1c0bebba44a64f664cb82544f212a7d073e81aeabae194095a3df1e48


enter position

In [24]:
farming_token_nonce, farming_token_amount, _ = get_token_details_for_address(proxy_contract.proxy_lp_token, user.address.bech32(), context.network_provider.proxy)
farm_token_nonce, farm_token_amount, _ = get_token_details_for_address(proxy_contract.proxy_farm_token, user.address.bech32(), context.network_provider.proxy)
if farm_token_nonce == 0:
    print("No farm tokens")
    exit()
event = DexProxyEnterFarmEvent(farm_contract, 
                               proxy_contract.proxy_lp_token, farming_token_nonce, farming_token_amount,
                               proxy_contract.proxy_farm_token, farm_token_nonce, farm_token_amount)
txhash = proxy_contract.enter_farm_proxy(user, context.network_provider.proxy, event)

2023-08-11 01:43:47,001 - DEBUG - Executing enter farm via proxy for user erd1ynecckmk8apsy086e49ng6mske566e8c5sc9val809z7rcth4j2sdal443 with event {'farmContract': <contracts.farm_contract.FarmContract object at 0x7f9dbe883310>, 'farming_tk': 'XMEXLP-2cbdce', 'farming_tk_nonce': 59828, 'farming_tk_amount': 26402952534061975757, 'farm_tk': 'XMEXFARM-e0a3dc', 'farm_tk_nonce': 193574, 'farm_tk_amount': 27219538694909253357}
2023-08-11 01:43:47,003 - DEBUG - enter farm via proxy
2023-08-11 01:43:47,427 - DEBUG - Contract call arguments: [erd1qqqqqqqqqqqqqpgqapxdp9gjxtg60mjwhle3n6h88zch9e7kkp2s8aqhkg]
2023-08-11 01:43:48,048 - INFO - No explorer known for https://proxy-shadowfork-one.elrond.ro. transaction raw path: transactions/48f8b24b3684d179a3e739426c49a416bfe503a747b0d7da3a9c4c969e592990


exit lp position

In [34]:
farming_token_nonce, farming_token_amount, _ = get_token_details_for_address(proxy_contract.proxy_lp_token, user.address.bech32(), context.network_provider.proxy)
if farm_token_nonce == 0:
    print("No tokens")
    exit()
event = DexProxyRemoveLiquidityEvent(pair_contract, farming_token_amount, farming_token_nonce, 1, 1)
txhash = proxy_contract.remove_liquidity_proxy(user, context.network_provider.proxy, event)

2023-08-09 19:28:52,737 - DEBUG - Executing remove liquidity via proxy for user erd1ynecckmk8apsy086e49ng6mske566e8c5sc9val809z7rcth4j2sdal443 with event {'pairContract': <contracts.pair_contract.PairContract object at 0x7fb6b9227670>, 'amount': 49717612028259933112, 'nonce': 59819, 'amountA': 1, 'amountB': 1}
2023-08-09 19:28:52,739 - DEBUG - remove liquidity via proxy
2023-08-09 19:28:53,339 - DEBUG - Contract call arguments: [erd1qqqqqqqqqqqqqpgqa0fsfshnff4n76jhcye6k7uvd7qacsq42jpsp6shh2, 1, 1]
2023-08-09 19:28:54,493 - INFO - No explorer known for https://proxy-shadowfork-one.elrond.ro. transaction raw path: transactions/080958180e0eefb482ad28b1457fbdaa70cc4263ea3cbfd8f83751ddc520afe5


create lp position

In [21]:
first_token_nonce, first_token_amount, _ = get_token_details_for_address(pair_contract.firstToken, user.address.bech32(), context.network_provider.proxy)

xmex = next(locked_token for locked_token in proxy_contract.locked_tokens if "XMEX" in locked_token)
xmex_list = get_all_token_nonces_details_for_account(xmex, user.address.bech32(), context.network_provider.proxy)
biggest_xmex = {}
for xmex_found in xmex_list:
    # get the highest amount one in the list
    if int(xmex_found['balance']) > int(biggest_xmex.get('balance', 0)):
        biggest_xmex = xmex_found
second_token_nonce, second_token_amount = biggest_xmex['nonce'], int(biggest_xmex['balance'])

if first_token_amount == 0 or second_token_amount == 0:
    print(f"No tokens: first token amount: {first_token_amount}; second token amount {second_token_amount}")
    exit()
event = DexProxyAddLiquidityEvent(pair_contract, pair_contract.firstToken, first_token_nonce, first_token_amount, 1,
                                  xmex, second_token_nonce, second_token_amount, 1,)
txhash = proxy_contract.add_liquidity_proxy(user, context.network_provider.proxy, event)

2023-08-11 01:41:04,661 - DEBUG - Executing add liquidity via proxy for user erd1ynecckmk8apsy086e49ng6mske566e8c5sc9val809z7rcth4j2sdal443 with event {'pairContract': <contracts.pair_contract.PairContract object at 0x7f9dbe8830a0>, 'tokenA': 'WEGLD-bd4d79', 'nonceA': 0, 'amountA': 23606332009211084887, 'amountAmin': 1, 'tokenB': 'XMEX-fda355', 'nonceB': 53, 'amountB': 135900053706923476104973053, 'amountBmin': 1}
2023-08-11 01:41:04,663 - DEBUG - add liquidity via proxy
2023-08-11 01:41:04,971 - DEBUG - Contract call arguments: [erd1qqqqqqqqqqqqqpgqa0fsfshnff4n76jhcye6k7uvd7qacsq42jpsp6shh2, 1, 1]
2023-08-11 01:41:05,553 - INFO - No explorer known for https://proxy-shadowfork-one.elrond.ro. transaction raw path: transactions/d1d88587fb099e0412f6f94acd83056dfa2e0ce8b5bef06f33c481bb0a48e887


exit position

In [18]:
farm_token_nonce, farm_token_amount, _ = get_token_details_for_address(proxy_contract.proxy_farm_token, user.address.bech32(), context.network_provider.proxy)
if farm_token_nonce == 0:
    print("No farm tokens")
    exit()
event = DexProxyExitFarmEvent(farm_contract, proxy_contract.proxy_farm_token, farm_token_nonce, farm_token_amount)
txhash = proxy_contract.exit_farm_proxy(user, context.network_provider.proxy, event)

2023-08-11 01:30:25,125 - DEBUG - Executing exit farm via proxy for user erd1ynecckmk8apsy086e49ng6mske566e8c5sc9val809z7rcth4j2sdal443 with event {'farmContract': <contracts.farm_contract.FarmContract object at 0x7f9dbe883310>, 'token': 'XMEXFARM-e0a3dc', 'nonce': 193571, 'amount': 10305923245858968100}
2023-08-11 01:30:25,127 - DEBUG - exit farm via proxy
2023-08-11 01:30:25,566 - DEBUG - Contract call arguments: [erd1qqqqqqqqqqqqqpgqapxdp9gjxtg60mjwhle3n6h88zch9e7kkp2s8aqhkg, 10305923245858968100]
2023-08-11 01:30:25,797 - INFO - No explorer known for https://proxy-shadowfork-one.elrond.ro. transaction raw path: transactions/e010f9dc717ade8d29762a784dd2cc65c78a74cd2c91619f3cd028bedbeaea13


Lock tokens

In [7]:
_, mex_amount, _ = get_token_details_for_address(proxy_contract.token, user.address.bech32(), context.network_provider.proxy)

lockable_tokens = ESDTToken(proxy_contract.token, 0, mex_amount)
txhash = energy_contract.lock_tokens(user, context.network_provider.proxy, [[lockable_tokens], 720])

2023-08-09 19:31:15,265 - INFO - lock tokens
2023-08-09 19:31:15,267 - DEBUG - lock tokens
2023-08-09 19:31:16,185 - DEBUG - Contract call arguments: [720]
2023-08-09 19:31:16,743 - INFO - No explorer known for https://proxy-shadowfork-one.elrond.ro. transaction raw path: transactions/3a237dcc6918e77b3a839890214e941bfc35bdf132ebf1d29c522cbce4ac14a7


create lp position

In [8]:
first_token_nonce, first_token_amount, _ = get_token_details_for_address(pair_contract.firstToken, user.address.bech32(), context.network_provider.proxy)

xmex = next(locked_token for locked_token in proxy_contract.locked_tokens if "XMEX" in locked_token)
xmex_list = get_all_token_nonces_details_for_account(xmex, user.address.bech32(), context.network_provider.proxy)
biggest_xmex = {}
for xmex_found in xmex_list:
    # get a specific xmex nonce from the list
    if int(xmex_found['nonce']) == 17:
        biggest_xmex = xmex_found
second_token_nonce, second_token_amount = biggest_xmex['nonce'], int(biggest_xmex['balance'])

if first_token_amount == 0 or second_token_amount == 0:
    print(f"No tokens: first token amount: {first_token_amount}; second token amount {second_token_amount}")
    exit()
event = DexProxyAddLiquidityEvent(pair_contract, pair_contract.firstToken, first_token_nonce, first_token_amount, 1,
                                  xmex, second_token_nonce, second_token_amount, 1,)
txhash = proxy_contract.add_liquidity_proxy(user, context.network_provider.proxy, event)

2023-08-09 19:33:41,735 - DEBUG - Executing add liquidity via proxy for user erd1ynecckmk8apsy086e49ng6mske566e8c5sc9val809z7rcth4j2sdal443 with event {'pairContract': <contracts.pair_contract.PairContract object at 0x7f9dbe8830a0>, 'tokenA': 'WEGLD-bd4d79', 'nonceA': 0, 'amountA': 29163864616900849236, 'amountAmin': 1, 'tokenB': 'XMEX-fda355', 'nonceB': 17, 'amountB': 31942150868735998483800174, 'amountBmin': 1}
2023-08-09 19:33:41,737 - DEBUG - add liquidity via proxy
2023-08-09 19:33:42,533 - DEBUG - Contract call arguments: [erd1qqqqqqqqqqqqqpgqa0fsfshnff4n76jhcye6k7uvd7qacsq42jpsp6shh2, 1, 1]
2023-08-09 19:33:42,919 - INFO - No explorer known for https://proxy-shadowfork-one.elrond.ro. transaction raw path: transactions/b174f539ce97eb8d3900b9895a8c18da3302cbe7c6f386cdea275973b5d7cfbd


enter new position

In [19]:
farming_token_nonce, farming_token_amount, _ = get_token_details_for_address(proxy_contract.proxy_lp_token, user.address.bech32(), context.network_provider.proxy)
event = DexProxyEnterFarmEvent(farm_contract, 
                               proxy_contract.proxy_lp_token, farming_token_nonce, farming_token_amount // 2,
                               "", 0, 0)
txhash = proxy_contract.enter_farm_proxy(user, context.network_provider.proxy, event)

2023-08-11 01:38:36,390 - DEBUG - Executing enter farm via proxy for user erd1ynecckmk8apsy086e49ng6mske566e8c5sc9val809z7rcth4j2sdal443 with event {'farmContract': <contracts.farm_contract.FarmContract object at 0x7f9dbe883310>, 'farming_tk': 'XMEXLP-2cbdce', 'farming_tk_nonce': 59824, 'farming_tk_amount': 4998372774241599528, 'farm_tk': '', 'farm_tk_nonce': 0, 'farm_tk_amount': 0}
2023-08-11 01:38:36,391 - DEBUG - enter farm via proxy
2023-08-11 01:38:36,925 - DEBUG - Contract call arguments: [erd1qqqqqqqqqqqqqpgqapxdp9gjxtg60mjwhle3n6h88zch9e7kkp2s8aqhkg]
2023-08-11 01:38:37,336 - INFO - No explorer known for https://proxy-shadowfork-one.elrond.ro. transaction raw path: transactions/b6ac89fc6996e223f78850f548003d92333b367391af6bb06b5cc35dfa8b1cee


merge position

In [22]:
farming_token_nonce, farming_token_amount, _ = get_token_details_for_address(proxy_contract.proxy_lp_token, user.address.bech32(), context.network_provider.proxy)
farm_token_nonce, farm_token_amount, _ = get_token_details_for_address(proxy_contract.proxy_farm_token, user.address.bech32(), context.network_provider.proxy)
if farm_token_nonce == 0:
    print("No farm tokens")
    exit()
event = DexProxyEnterFarmEvent(farm_contract, 
                               proxy_contract.proxy_lp_token, farming_token_nonce, farming_token_amount,
                               proxy_contract.proxy_farm_token, farm_token_nonce, farm_token_amount)
txhash = proxy_contract.enter_farm_proxy(user, context.network_provider.proxy, event)

2023-08-11 01:41:46,303 - DEBUG - Executing enter farm via proxy for user erd1ynecckmk8apsy086e49ng6mske566e8c5sc9val809z7rcth4j2sdal443 with event {'farmContract': <contracts.farm_contract.FarmContract object at 0x7f9dbe883310>, 'farming_tk': 'XMEXLP-2cbdce', 'farming_tk_nonce': 59826, 'farming_tk_amount': 44442331841335307657, 'farm_tk': 'XMEXFARM-e0a3dc', 'farm_tk_nonce': 193573, 'farm_tk_amount': 9996745548483199057}
2023-08-11 01:41:46,305 - DEBUG - enter farm via proxy
2023-08-11 01:41:46,734 - DEBUG - Contract call arguments: [erd1qqqqqqqqqqqqqpgqapxdp9gjxtg60mjwhle3n6h88zch9e7kkp2s8aqhkg]
2023-08-11 01:41:47,144 - INFO - No explorer known for https://proxy-shadowfork-one.elrond.ro. transaction raw path: transactions/e373fc0147efdaa3ceb2179b6b83c971418453c47097b0cbce7bafdb1e8ea9fe


In [ ]:
from tools.account_state import get_account_keys_online, report_key_files_compare
get_account_keys_online(energy_contract.address, context.network_provider.proxy.url, with_save_in="state_dump/pre_upgrade_energy_factory.json")

In [ ]:
get_account_keys_online(energy_contract.address, context.network_provider.proxy.url, with_save_in="state_dump/post_upgrade_energy_factory.json")
report_key_files_compare("state_dump", "pre", "post", True)